In [1]:
import pandas as pd
import json
from xopen import xopen


In [2]:
predctions_path = "../qa_predictions/20_total_documents/"

In [3]:
llama_regular = predctions_path + "gpt/"
file_names = [f"{llama_regular}nq-open-20_total_documents_gold_at_{i}-gpt-35-reorder-predictions.jsonl.gz" for i in [0,4,9,14,19]]

In [4]:
position_0 = file_names[0]
position_4 = file_names[1]
position_9 = file_names[2]
position_14 = file_names[3]
position_19 = file_names[4]


In [7]:
predictions = []
for file in file_names:
    data = []
    with xopen(file, "r") as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except:
                print(line[:10])
    print(len(data))
    predictions.append(data)

2655
2655
2655
2655
2655


In [8]:
for i in range(len(predictions)):
    file = file_names[i]
    print(file)
    data = predictions[i]
    with xopen(file, "w") as clean_json:
        for input_example in list(data)[:2655]:
            clean_json.write(json.dumps(input_example) + "\n")



../qa_predictions/20_total_documents/gpt/nq-open-20_total_documents_gold_at_0-gpt-35-reorder-predictions.jsonl.gz
../qa_predictions/20_total_documents/gpt/nq-open-20_total_documents_gold_at_4-gpt-35-reorder-predictions.jsonl.gz
../qa_predictions/20_total_documents/gpt/nq-open-20_total_documents_gold_at_9-gpt-35-reorder-predictions.jsonl.gz
../qa_predictions/20_total_documents/gpt/nq-open-20_total_documents_gold_at_14-gpt-35-reorder-predictions.jsonl.gz
../qa_predictions/20_total_documents/gpt/nq-open-20_total_documents_gold_at_19-gpt-35-reorder-predictions.jsonl.gz


In [9]:
file = file_names[1]
print(file)
data = predictions[1]
with xopen(file, "w") as clean_json:
    for input_example in list(data)[:2655]:
        clean_json.write(json.dumps(input_example) + "\n")



../qa_predictions/20_total_documents/gpt/nq-open-20_total_documents_gold_at_4-gpt-35-reorder-predictions.jsonl.gz


In [11]:
all_questions = set()
for i, dataset in enumerate(predictions):
    tmp = set()
    for d in dataset:
        small_dict = {"question": d["question"], "ctxs": d["ctxs"]}
        tmp.add(json.dumps(small_dict))
        all_questions.add(d["question"])
    print(f"data {i}: {len(tmp)}")

data 0: 2655
data 1: 2695
data 2: 2655
data 3: 2655
data 4: 2655


In [12]:
len(all_questions)

2655

# Find missing question and add responses

In [129]:
import sys
import time
from copy import deepcopy
import dataclasses
from huggingface_hub import InferenceClient

sys.path.append("../src")

In [130]:
from lost_in_the_middle.prompting import get_qa_prompt, Document

In [131]:
# Find missing questions
tmp_qa_set = set()
for d in predictions[1]:
    tmp_qa_set.add(d["question"])
print(len(tmp_qa_set))
with xopen(f"../qa_data/20_total_documents/nq-open-20_total_documents_gold_at_4.jsonl.gz", "r") as f:
    original_qa_set = set()
    data_dict = {}
    for line in f:
        data = json.loads(line)
        data_dict[f"{data['question']}"] = data
        original_qa_set.add(data["question"])
    
# Find missing questions
print(len(original_qa_set), len(tmp_qa_set))
missing_questions = original_qa_set - tmp_qa_set
missing_questions

2653
2655 2653


{'when did computer become widespread in homes and schools',
 'when was as you like it first performed'}

In [132]:
missing_data = [data_dict[missing] for missing in missing_questions]

In [133]:
HF_HUB_MODELS = {
    "llama-2-70b": "meta-llama/Llama-2-70b-chat-hf",
    "llama-2-13b": "meta-llama/Llama-2-13b-chat-hf",
    "llama-2-7b": "meta-llama/Llama-2-7b-chat-hf",
}
HUGGINGFACEHUB_API_TOKEN = "YOUR_TOKEN"

In [134]:
output_path = file_names[1]
model_name = "llama-2-7b"
llm = InferenceClient(
        model=HF_HUB_MODELS[model_name],
        token=HUGGINGFACEHUB_API_TOKEN,
    )

# Get results from the missing data
prompts = []
examples = []
all_model_documents = []
for input_example in list(missing_data):
    # Get the prediction for the input example
    question = input_example["question"]

    documents = []
    for ctx in deepcopy(input_example["ctxs"]):
        documents.append(Document.from_dict(ctx))
    if not documents:
        raise ValueError(f"Did not find any documents for example: {input_example}")

    
    prompt = get_qa_prompt(
        question,
        documents,
        mention_random_ordering=False,
        query_aware_contextualization=True,
        cot_prompting=False,
        chain_prompting=False,
        tot_prompting=False,
        llama_prompting=True,
    )

    
    prompts.append(prompt)
    examples.append(deepcopy(input_example))
    all_model_documents.append(documents)

responses = []
missing_responses = []
with xopen(output_path, "a") as f:
    for i in range(len(prompts)):
        response = ""
        example, model_documents, prompt = examples[i], all_model_documents[i], prompts[i]
        count = 0
        sleep_time = 1
        while len(response) == 0 and count < 5:
            try:
                response = llm.text_generation(prompt, max_new_tokens=100, temperature=0.6)
            except Exception as e:
                time.sleep(sleep_time)
                count += 1
                sleep_time *= 2
        if len(response) == 0:
            missing_responses.append(prompt)
            response = "Error generating response"
        else:
            responses.append(response)

        print(len(missing_responses))
        print(response)
        output_example = deepcopy(example)
        # Add some extra metadata to the output example
        output_example["model_prompt"] = prompt
        output_example["model_documents"] = [dataclasses.asdict(document) for document in model_documents]
        output_example["model_answer"] = response
        output_example["model"] = model_name
        output_example["model_temperature"] = 0.6
        output_example["model_top_p"] = 0
        output_example["model_prompt_mention_random_ordering"] = False
        output_example["model_use_random_ordering"] = False
        f.write(json.dumps(output_example) + "\n")
print(responses[0])
print(prompts[0])

0
  Based on the provided search results, it appears that computers started becoming widespread in homes and schools in the late 1970s to early 1980s. Here are some key points from the search results:

1. In 1975, Apple Inc. began donating Apple 1 models to schools, marking the beginning of home computing (Document [3](Title: Computers in the classroom)).
2. By 
0
  Based on the provided documents, it seems that "As You Like It" was first performed in the early 17th century, with the exact date of performance being uncertain.

Document [1](Title: As You Like It) states that the play was performed at the Old Vic Theatre in 1936, starring Laurence Olivier as Orlando and Elisabeth Bergner as Rosalind. However, this document does not provide any information about the play's
  Based on the provided search results, it appears that computers started becoming widespread in homes and schools in the late 1970s to early 1980s. Here are some key points from the search results:

1. In 1975, Apple I

In [136]:
responses

['  Based on the provided search results, it appears that computers started becoming widespread in homes and schools in the late 1970s to early 1980s. Here are some key points from the search results:\n\n1. In 1975, Apple Inc. began donating Apple 1 models to schools, marking the beginning of home computing (Document [3](Title: Computers in the classroom)).\n2. By ',
 '  Based on the provided documents, it seems that "As You Like It" was first performed in the early 17th century, with the exact date of performance being uncertain.\n\nDocument [1](Title: As You Like It) states that the play was performed at the Old Vic Theatre in 1936, starring Laurence Olivier as Orlando and Elisabeth Bergner as Rosalind. However, this document does not provide any information about the play\'s']